# Cell-Annotation

###  Cell annotation by large language model GPT-4
Hou, W., Ji, Z. Assessing GPT-4 for cell type annotation in single-cell RNA-seq analysis. Nat Methods 21, 1462–1465 (2024). https://doi.org/10.1038/s41592-024-02235-4

In [2]:
import scanpy as sc
from cell_annotator import CellAnnotator
import pandas as pd
import numpy as np
import glob
import os
import tqdm

## Load Gene Orthology Reference


In [3]:
# Load gene otholog reference table (adapted with `gene_reference.ipynb`)
gene_reference = pd.read_csv('/home/raquelcr/scanpy/gene_reference.csv')
gene_reference.head()

,NMR gene name,gene_ids,Human gene stable ID,Human gene name,Mouse gene stable ID,Mouse gene name,Local_ID,Global_name,Global_ID
0,ZMYND10,ENSHGLG00000010267,ENSG00000004838,ZMYND10,ENSMUSG00000010044,Zmynd10,LOCAL00000000000,ZMYND10,ENSHGLG00000010267
1,AMIGO3,ENSHGLG00000010542,NaN,NaN,NaN,NaN,LOCAL00000000001,AMIGO3,ENSHGLG00000010542
2,TNFSF10,ENSHGLG00000010558,NaN,NaN,NaN,NaN,LOCAL00000000002,TNFSF10,ENSHGLG00000010558
3,ENSHGLG00000010617,ENSHGLG00000010617,NaN,NaN,NaN,NaN,LOCAL00000000003,ENSHGLG00000010617,ENSHGLG00000010617
4,NPRL2,ENSHGLG00000010655,ENSG00000114388,NPRL2,ENSMUSG00000010057,Nprl2,LOCAL00000000004,NPRL2,ENSHGLG00000010655


In [9]:
nmr2human = pd.read_csv('/home/raquelcr/NMR-snRNA-seq/auto_annotation/nmr_to_human.csv')
nmr2human = nmr2human[nmr2human['perc_id']>60] # filter out by perc_id
nmr2human

,heterocephalus_glaber_female_gene,homo_sapiens_gene,orthology_type,perc_id,dn_ds
0,U6,ENSG00000252980,ortholog_one2one,67.0000,NaN
1,C3orf38,ENSG00000179021,ortholog_one2one,85.1064,NaN
2,ZNF654,ENSG00000175105,ortholog_one2one,90.5142,NaN
3,CGGBP1,ENSG00000163320,ortholog_one2one,99.4012,NaN
4,HTR1F,ENSG00000179097,ortholog_one2one,93.9891,NaN
...,...,...,...,...,...
12378,COX2,ENSG00000198712,ortholog_one2one,69.1630,NaN
12380,ND3,ENSG00000198840,ortholog_one2one,67.8261,NaN
12381,ND4,ENSG00000198886,ortholog_one2one,69.4989,NaN
12382,ND5,ENSG00000198786,ortholog_one2one,65.1741,NaN


In [ ]:
# Map mouse gene names to global names using gene_reference
mouse_gene_map = dict(zip(gene_reference['Mouse gene name'], gene_reference['Global_name']))
mouse_data.var['global_name'] = mouse_data.var_names.map(mouse_gene_map)


## Load NMR data 

In [4]:
nmr_files = glob.glob('/home/raquelcr/scanpy/cellbender/denoised/*_filtered.h5')
nmr_files

['/home/raquelcr/scanpy/cellbender/denoised/NMR2_cerebral_cortex_denoised_filtered.h5',
 '/home/raquelcr/scanpy/cellbender/denoised/NMR1_cerebral_cortex_denoised_filtered.h5',
 '/home/raquelcr/scanpy/cellbender/denoised/NMR3_hippocampus_denoised_filtered.h5',
 '/home/raquelcr/scanpy/cellbender/denoised/NMR4_hippocampus_denoised_filtered.h5',
 '/home/raquelcr/scanpy/cellbender/denoised/NMR5_midbrain_denoised_filtered.h5',
 '/home/raquelcr/scanpy/cellbender/denoised/NMR6_midbrain_denoised_filtered.h5']

In [5]:
nmr_adatas = []
nmr_gene_map = dict(zip(gene_reference['NMR gene name'], gene_reference['Global_name']))

for file in nmr_files:
    # CellBender output should be read with read_10x_h5
    try: 
        adata = sc.read_10x_h5(file)
    except:
        adata = sc.read(file)

    adata.var['species'] = 'nmr'

    sample = file.replace('/home/raquelcr/scanpy/cellbender/denoised/NMR', '').replace('_denoised_filtered.h5', '').split('_', 1)
    sample_id = sample[0]
    tissue = sample[1]

    print(f'Loading sample {sample_id} of {tissue}')

    adata.obs['species'] = 'nmr'
    adata.obs['tissue'] = tissue
    adata.obs['sample_id'] = sample_id
    adata.obs['replicate'] = (int(sample_id)+1)%2+1

    # Map NMR gene IDs to global names
    adata.var['global_name'] = adata.var_names.map(nmr_gene_map)
    
    nmr_adatas.append(adata)

nmr_adatas

Loading sample 2 of cerebral_cortex
Loading sample 1 of cerebral_cortex
Loading sample 3 of hippocampus
Loading sample 4 of hippocampus
Loading sample 5 of midbrain
Loading sample 6 of midbrain


[AnnData object with n_obs × n_vars = 19487 × 20774
     obs: 'species', 'tissue', 'sample_id', 'replicate'
     var: 'gene_ids', 'feature_types', 'genome', 'species', 'global_name',
 AnnData object with n_obs × n_vars = 14268 × 20774
     obs: 'species', 'tissue', 'sample_id', 'replicate'
     var: 'gene_ids', 'feature_types', 'genome', 'species', 'global_name',
 AnnData object with n_obs × n_vars = 6743 × 20774
     obs: 'species', 'tissue', 'sample_id', 'replicate'
     var: 'gene_ids', 'feature_types', 'genome', 'species', 'global_name',
 AnnData object with n_obs × n_vars = 8179 × 20774
     obs: 'species', 'tissue', 'sample_id', 'replicate'
     var: 'gene_ids', 'feature_types', 'genome', 'species', 'global_name',
 AnnData object with n_obs × n_vars = 7369 × 20774
     obs: 'species', 'tissue', 'sample_id', 'replicate'
     var: 'gene_ids', 'feature_types', 'genome', 'species', 'global_name',
 AnnData object with n_obs × n_vars = 10080 × 20774
     obs: 'species', 'tissue', 'samp

In [ ]:
# Change gene names to human gene names


### Load few cells from human (adult and developing human) to perform cross-validation by triangulation

## GPT-Cell-Annotation

In [ ]:
cell_ann = CellAnnotator(
    adata, species="human", tissue="brain", cluster_key="leiden", sample_key="samples",
).annotate_clusters()